In [ ]:
from ftplib import FTP
from zipfile import ZipFile
from PIL import Image
import glob

### Download data from PRISM FTP

In [ ]:
data_dir = r"C:\Users\msong\Desktop\arc2\lab2\lab2"
out_dir = r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_bil_data"

In [ ]:
# Access and navigate prism ftp server to data
ftp = FTP("prism.nacse.org")
ftp.login(user ='anonymous', passwd = 'leex6165@umn.edu' )
ftp.cwd('normals_4km')
ftp.cwd('ppt')

In [ ]:
# create list of all data files in ppt data folder 
all_normal_data = ftp.nlst()

In [ ]:
# extract names of bil files in ppt data folder
normal_bil = []
for item in all_normal_data:
    if ("all" in item) and ("bil" in item):
        normal_bil.append(item)

In [ ]:
# download all bil zipped files from ftp
for item in normal_bil:
    with open(item, "wb") as file:
        ftp.retrbinary("RETR " + item, file.write, 1024)
    
    with ZipFile(f"{data_dir}/{item}") as zip_fldr:
        zip_fldr.extractall(out_dir)

## Create space time cube of annual precitation

In [ ]:
# create a mosaic dataset
in_workspace = r"C:\Users\msong\Desktop\arc2\lab2\lab2\lab2.gdb"
in_mosaicdataset_name = "prism_30ppt"
coordinate_system = "GEOGCS['GCS_North_American_1983',DATUM['D_North_American_1983',SPHEROID['GRS_1980',6378137.0,298.257222101]],PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]]"

arcpy.management.CreateMosaicDataset(in_workspace,
                             in_mosaicdataset_name, 
                             coordinate_system, 
                             1,  
                             "", 
                             "NONE", 
                             None)

In [ ]:
# convert bil rasters to tiff to create space time cube
arcpy.env.workspace = r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_bil_data"
for bil in arcpy.ListRasters("*", "bil"):
    arcpy.conversion.RasterToOtherFormat(bil,
                                         out_dir, 
                                         "TIFF")   

In [ ]:
# resample tif to 8kx8k meters for performance
arcpy.env.workspace = r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_bil_data"
for tif in arcpy.ListRasters("*", "tif"):
    arcpy.management.Resample(tif, 
                              f"{out_dir}/resample/ppt_8k_{tif[-10:-8]}.tif", 
                              "0.08 0.08", 
                              "NEAREST")

In [ ]:
# add resampled tiff rasters to mosaic dataset
arcpy.env.workspace = r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_bil_data\resample"
for tif in arcpy.ListRasters("*", "tif"):
    print(tif)
    if "ppt_8k_al" in tif:
        pass
    else:
        arcpy.management.AddRastersToMosaicDataset(r"C:\Users\msong\Desktop\arc2\lab2\lab2\lab2.gdb\prism_30ppt", 
                                                   "Raster Dataset", 
                                                   tif)

In [ ]:
# add new variable field and populate with precipitation
var_field = "variable"
ppt_field = "timestamp"
arcpy.env.workspace = r"C:\Users\msong\Desktop\arc2\lab2\lab2\lab2.gdb"

arcpy.AddField_management(in_mosaicdataset_name, var_field, "TEXT")
arcpy.management.CalculateField(in_mosaicdataset_name, 
                                var_field, 
                                '"precipitation"', 
                                "PYTHON3", 
                                '', 
                                "TEXT")



# Add timestamp field and populate with months in 2010
arcpy.AddField_management(in_mosaicdataset_name, ppt_field, "DATE")
arcpy.management.CalculateField(in_mosaicdataset_name, 
                                ppt_field, 
                                'DateAdd(Date(2010,0,1), $feature.OBJECTID-1, "month")', 
                                "ARCADE", 
                                '', 
                                "TEXT")

In [ ]:
# Format variable and time to prepare multidimensional raster
arcpy.md.BuildMultidimensionalInfo(in_mosaicdataset_name, 
                                   var_field, 
                                   "timestamp # #", 
                                   "precipitation # #")

In [ ]:
arcpy.env.overwriteOutput = True

In [ ]:
# make multidimensional raster layer
arcpy.md.MakeMultidimensionalRasterLayer("prism_30ppt", 
                                         "prism_30ppt_MultidimLayer", 
                                         "precipitation", 
                                         "ALL", 
                                         None, 
                                         None, 
                                         '', 
                                         '', 
                                         '', 
                                         None, 
                                         '', 
                                         "-125.020833333 24.0975 -66.460833333 49.9375", 
                                         "DIMENSIONS")

In [ ]:
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer("prism_30ppt_MultidimLayer", 
                                            r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_annual30ppt_tc.nc", 
                                            "ZEROS")

In [ ]:
arcpy.stpm.VisualizeSpaceTimeCube3D(r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_annual30ppt_tc.nc", 
                                    "PRECIPITATION_NONE_ZEROS", 
                                    "VALUE", 
                                    r"C:\Users\msong\Desktop\arc2\lab2\lab2\lab2.gdb\prism_annual30ppt_tc2_3D")

In [ ]:
# Repeat this 12 times for each image
# I used the multidimensional layer to do this and exported the map manually
# Couldn't figure out how to export map project to jpeg through arcpy

# exports layout to jpeg
aprx = arcpy.mp.ArcGISProject("CURRENT")
mxd_list = aprx.listLayouts("Layout3")[0]

outpath= r"C:\Users\msong\Desktop"
layout = mxd_list.exportToJPEG(f"{outpath}/prism_gif/01.jpg")

In [ ]:
# I ran this in the jupyter notebooks not in the arcpro environment
# Create the frames
frames = []
imgs = glob.glob(r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_gif\*.jpg")
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
    
# Save GIF
frames[0].save(r"C:\Users\msong\Desktop\arc2\lab2\lab2\prism_gif\ppt_gif.gif", format="GIF",
               append_images=frames[1:],
               save_all=True,
               duration=300, loop=0)